# VegET Testing

Imports

In [1]:
import datetime
import dateutil.parser
import VegET
from VegET import interpolate, daily_aggregate, utils, veg_et_model
#import cartoee as cee
import matplotlib.pyplot as plt
import bqplot
import ipyleaflet
import IPython.display
import numpy as np
import pandas as pd
import traitlets
import ee
import ipywidgets as widgets
import ipyleaflet  # an interactive mapping "widget"
from sidecar import Sidecar
# from ipygee import *
import ipygee

Initialize EarthEngine

In [2]:
ee.Initialize()

Define date range

In [3]:
start_date = ee.Date('2003-04-01')
end_date = ee.Date('2003-11-01')

Define ROI 

In [4]:
# ROI
roi_fc = ee.FeatureCollection('EPA/Ecoregions/2013/L4');
polygon = roi_fc.filter(ee.Filter.eq('system:index', '00000a53e3e196f3200c'))

# Filter to only include images within the colorado and utah boundaries (from ee-api/python examples)
# polygon = ee.Geometry.Polygon([[
#     [-109.05, 37.0], [-102.05, 37.0], [-102.05, 41.0],   # colorado
#     [-109.05, 41.0], [-111.05, 41.0], [-111.05, 42.0],   # utah
#     [-114.05, 42.0], [-114.05, 37.0], [-109.05, 37.0]]])

Define growing season months as integers. Note, filtering is inclusive.

In [5]:
g_season_begin = 4
g_season_end = 10

**NOTE**: for this case, the imagecollections are global or continent wide rasters. Ordinarily, the
imageCollections would need `.filterBounds()` to the ROI to subset to the images that intersect the
polygon. In this case, the filter does nothing since the images are continent/global scale.

Get ImageCollection used to calculate NDVI values. In this example, MODIS data are used.

In [6]:
ndvi_coll = ee.ImageCollection("MODIS/006/MOD09Q1").filterDate(start_date, end_date)\
    .filter(ee.Filter.calendarRange(g_season_begin, g_season_end, 'month'))\
    .map(lambda f: f.clip(polygon))
ndvi_coll = ndvi_coll.map(VegET.utils.getNDVI)

Get daily climate data (precip, eto, temp)

In [7]:
precip_eto_coll = ee.ImageCollection('IDAHO_EPSCOR/GRIDMET').filterDate(start_date, end_date)\
    .select('pr', 'eto', 'tmmn', 'tmmx').filter(ee.Filter.calendarRange(g_season_begin, g_season_end, 'month'))\
    .map(lambda f: f.clip(polygon))

# Add band for calculated mean daily temp
precip_eto_coll = precip_eto_coll.map(VegET.utils.dailyMeanTemp)
# Convert to Celsius
precip_eto_coll = precip_eto_coll.map(VegET.utils.kelvin2celsius).select(['pr', 'eto', 'tminC', 'tmaxC', 'tmeanC'])

VegET static inputs

In [8]:
# Specify canopy intercept image or imageCollection. NOTE: Assumes single band image
canopy_int = ee.Image('users/darin_EE/VegET/Interception').clip(polygon).double().rename('intercept')
# Get static Soil Water Holding Capacity grid (manually uploaded as GEE asset)
whc = ee.Image('users/darin_EE/VegET/WaterHoldingCapacity_mm').clip(polygon).double().rename('whc')
# Get static Soil Saturation image
soil_sat = ee.Image('users/darin_EE/VegET/SoilSaturation_mm').clip(polygon).double().rename('soil_sat')
# Get static Field Capacity image
fcap = ee.Image('users/darin_EE/VegET/FieldCapacity_mm').clip(polygon).double().rename('fcap')

# Create single static image with static inputs as bands
staticImage = canopy_int.addBands([whc, soil_sat, fcap])

Add static data to ndvi_coll as bands

In [9]:
ndvi_coll = ndvi_coll.map(VegET.utils.addStaticBands([staticImage]))

Daily interpolation. Primarily using methods developed in OpenET

In [10]:
# Create daily interpolated ndvi collection
ndvi_daily = interpolate.daily(precip_eto_coll, ndvi_coll)

# Add date band as 'time'
ndvi_daily = ee.ImageCollection(ndvi_daily.map(VegET.utils.add_date_band))

## Run VegET model

In [11]:
vegET_run = veg_et_model.vegET_model(ndvi_daily, polygon)

In [12]:
# image = vegET_run.first()
# print(image.bandNames().getInfo())

In [13]:
def GetTileLayerUrl(ee_image_object):
  map_id = ee.Image(ee_image_object).getMapId()
  tile_url_template = "https://earthengine.googleapis.com/map/{mapid}/{{z}}/{{x}}/{{y}}?token={token}"
  return tile_url_template.format(**map_id)

In [14]:
vis_params = {
    'bands': ['swf', 'pr'],
    'min':0.0,
    'max':50.0,
}

In [15]:
roi_dimension = widgets.IntSlider(
    value=1e4,
    min=1e2,
    max=2e4,
    description='ROI Size (m):',
    continuous_update=False,
)

In [16]:
# Define the map.
map1 = ipygee.map.Map()
# map1.add_control(ipyleaflet.LayersControl())
map1.show()
# Define and add a Marker pin to the map.
center_marker = ipyleaflet.Marker(
    name='ROI Selection Marker',
    location=map1.center
)
map1 += center_marker

# veget_layer_group = ipyleaflet.LayerGroup(layers = (), name = 'VegET Results')
# map1 += veget_layer_group
# mosaic_layer_group = ipyleaflet.LayerGroup(layers=(), name='VegET Layer')
# map1 += mosaic_layer_group

roi_layer_group = ipyleaflet.LayerGroup(layers=(), name='ROI Layer')
map1 += roi_layer_group

Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map …

In [17]:
start_datepicker = widgets.DatePicker(
    description='Start Date',
    disabled=False,
    value=datetime.datetime(2003, 4, 8)
)
end_datepicker = widgets.DatePicker(
    description='End Date',
    disabled=False,
    value=datetime.datetime(2003, 4, 30)
)

In [19]:
out = widgets.Output()

In [ ]:
# Layout the UI elements.
panel = widgets.VBox([
    map1,
    start_datepicker,
    end_datepicker,
    roi_dimension,
    out
])

out.clear_output()
display(panel)

In [ ]:
# Define helper functions to swap the coordinate ordering.
def swap_coordinate_xy_for_location(coord):
    return (coord[1],coord[0])

def swap_coordinate_xy_for_list(coord_list):
    return [swap_coordinate_xy_for_location(coord) for coord in coord_list]  

def update_roi_layer(map_reference):    
    coord_list_xy = get_roi_polygon()['coordinates'][0]
    coord_list_yx = swap_coordinate_xy_for_list(coord_list_xy)
    
    roi_layer = ipyleaflet.Polygon(
        name='TEST update ROI Polygon',
        locations=coord_list_yx,
        weight=3,
        color='#F00',
        opacity=0.8,
        fill_opacity=0.1,
        fill_color='#F00'
    )
    roi_layer_group.clear_layers()
    roi_layer_group.add_layer(roi_layer)

In [ ]:
def get_roi_polygon():
    center_marker_xy = swap_coordinate_xy_for_location(center_marker.location)
    centroid = ee.Geometry.Point(center_marker_xy)
    buffered = centroid.buffer(roi_dimension.value).bounds()
    return buffered.getInfo() 

In [ ]:
print(get_roi_polygon())

In [ ]:
def get_image_collection():

    # Get filter values from the UI widgets.
    roi = get_roi_polygon()
    start_date = ee.Date(start_datepicker.value.isoformat())
    end_date = ee.Date(end_datepicker.value.isoformat())

    collection = (
        base_collection
          .filterDate(start_date, end_date).map(lambda f: f.clip(roi))
    )
    
    return collection

In [ ]:
base_collection = vegET_run

In [ ]:
def update_veget_layers(map_reference):
    
    out.clear_output()
    with out:
        print('Total images = {0}'.format(get_image_collection().size().getInfo()))
    
#     mosaic_tilelayer = ipyleaflet.TileLayer(
        
#         url=GetTileLayerUrl(
#             get_image_collection().first().visualize(**vis_params)
#         ),
#         attribution='Map tiles by <a href="http://earthengine.google.com/">Earth Engine</a>.'
#     )
    
#     map1.clear_layers()
    map_reference.addImageCollection(get_image_collection())

In [ ]:
# Define the actions performed when the marker moves.
def center_marker_on_move(change):
    update_roi_layer(map1)
    update_mosaic_layer(map1)
center_marker.unobserve_all()
center_marker.observe(center_marker_on_move, names='location')

In [ ]:
# Define the actions performed when the ROI size is changed.
def roi_dimension_on_change(change):
    update_roi_layer(map1)
    update_mosaic_layer(map1)
roi_dimension.unobserve_all()
roi_dimension.observe(roi_dimension_on_change, names='value')

In [ ]:
update_roi_layer(map1)
update_veget_layers(map1)

# Testing out a series

This section will demonstrate outputing an image time series for the specified location, time interval, and image collection.

In [ ]:
collection = get_image_collection()
print(collection.size().getInfo())


In [ ]:
MAX_ELEMENTS=100
images = collection.toList(MAX_ELEMENTS).getInfo()

In [ ]:
import os
output_directory = 'vegET_output'
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

## leaflet with tile_url

In [ ]:
map1 = ipyleaflet.Map(
    center=(46.33412852950776, -102.35768788380938), zoom=4,
    layout={'height':'400px'},
)
map1

In [ ]:
test_tilelayer = ipyleaflet.TileLayer(
    name='VegET first',
    url=GetTileLayerUrl(
        image.visualize(
            min=0,
            max=20,
            gamma=1.5,
            bands= ['swf']
        )
    ),
    attribution='Map tiles by <a href="http://earthengine.google.com/">Earth Engine</a>.'
)
map1.add_layer(test_tilelayer)

In [ ]:
# Adding the layers control to the map.
map1.add_control(ipyleaflet.LayersControl())

## ipygee: Create a Map instance    

Arguments:   
- tabs: a tuple indicating which tabs to load in the map. Options are: Inspector, Layers, Assets, Tasks   
- kwargs: as this class inherits from ipyleaflet. Map it can accept all its arguments

In [ ]:
Map = Map()

## Show map with method show
- Arguments
    - tabs: show tabs (bool)
    - layer_control: show a control for layers (bool)
    - draw_control: show a control for drawings (bool)
    - fullscrean: show fullscreen button (bool)

In [ ]:
Map.show()

## Resize Map
Dimensions must be in pixel units

In [ ]:
Map.setDimensions('75%', '1000px')

## Define visualization parameters

In [ ]:
visParam = {'bands': ['swi', 'swe', 'snowpack'], 'min': 0, 'max': 100}
visParamSwf = {'bands': ['swf'], 'min': 0, 'max': 80}

## Add Layers   

In [ ]:
Map.addLayer(image, visParam, name = 'VegET first')
#Map.addLayer(polygon, name = 'ROI')

## Add multiple images from a collection   

In [ ]:
Map.addImageCollection(vegET_run.limit(20), visParamSwf, namePattern = 'VegET results for {system_date}')

## Timeseries plots   

In [ ]:
test_site = ee.Geometry.Point([-102.35768788380938, 46.33412852950776])
test_feat = ee.Feature(test_site, {'name': 'test feature', 'buffer': 0})
bands = ['swf', 'pr']

testColl = vegET_run.filterDate('2003-04-10', '2003-09-30').select(bands)

chart_ts = chart.Image.series(**{
    'imageCollection': testColl,
    'region': test_feat,
    'scale': 250,
    'bands': bands,
    'label_bands': bands#,
#     'properties': ['system:index'],
#     'label_properties': ['index']
})

In [ ]:
chart_ts.renderWidget(width = '75%')